<a href="https://colab.research.google.com/github/GaboRamalho/Big-Data/blob/main/Processamento_e_An%C3%A1lise_Descritiva_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
# Iniciar uma sessão local e importar dados
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()

# **BASE DE DADOS 1**

## Explorando a base de dados

In [ ]:
df = sc.read.csv('terremotos.csv', header=True, sep=";",  inferSchema=True)
df.show(5)

In [ ]:
df.show(5, truncate=False)

### Visualizando as colunas

In [ ]:
df.columns

<a id='dataframe-schema'></a>
### Dataframe Schema

In [ ]:
df.dtypes

NameError: ignored

In [ ]:
df.printSchema()

## Colunas



1.   Selecionar coluna
2.   Selecionar várias colunas
3.   Adicionar coluna
4.   Renomear
5.   Agrupar
6.   Remover




### Selecionando 1 coluna

In [ ]:
df.select(df.Magnitude).show(10)

In [ ]:
from pyspark.sql.functions import col
df.select(col('Magnitude')).show(10)


### Selecionando múltiplas colunas

In [ ]:
df.select(col('Magnitude'),col('Profundidade(km)')).show(10)


### Adicionar nova coluna

#### Datas

In [ ]:
from pyspark.sql.functions import *
df = df.withColumn('Hora_', hour(df.Hora))
df = df.withColumn('Minuto', minute(df.Hora))
df.show(3, truncate=False)

In [ ]:
df = df.withColumn('Ano', year(to_date(df.Data, 'yyyy.MM.dd')))
df = df.withColumn('Mês', month(to_date(df.Data, 'yyyy.MM.dd')))
df.show(3, truncate=False)

In [ ]:
df = df.withColumn('Mult', (col('Profundidade(km)')*col('Magnitude')))
df.show(3)


### Renomear Coluna

In [ ]:
df = df.withColumnRenamed('Tipo', 'TipoTerremoto')
df.show(3, truncate=False)


### Agrupando por colunas

In [ ]:
df.groupBy('Ano').count().show(10)

In [ ]:
df.groupBy('TipoTerremoto').count().show()

In [ ]:
df.groupBy('TipoTerremoto', 'Ano').count().show(20)


### Remover Colunas

In [ ]:
df = df.drop('Data','Hora')
df.show(5,truncate=False)


## Linhas


1. Filtrando Linhas
2. Obtedo linhas distintas
3. Classificando Linhas





### Filtrando/selecionando por linhas

In [ ]:
total_contagem = df.count()
print("TOTAL Contagem: " + str(total_contagem)+'\n')

ano_2022_contagem = df.filter(col('Ano')==2022).count()
print("2022 Contagem: " + str(ano_2022_contagem)+'\n')

base_2022 = df.filter(col('Ano')==2022).show(truncate=False)

In [ ]:
ano_2022_magnitude = df.filter((col('Ano')==2022) &
                                  (col('Magnitude')>4)).count()

print("2022 Contagem:" + str(ano_2022_magnitude)+'\n')

df.filter((col('Ano')==2022) &
                              (col('Magnitude')>4)).show(truncate=False)


### Distinct

In [ ]:
df.select('Ano').distinct().show()

In [ ]:
df.select('Ano','TipoTerremoto').distinct().show()


### Classificando linhas

In [ ]:
df.orderBy('Ano').show(truncate=False)

In [ ]:

df.groupBy("Ano").count().orderBy('count', ascending=False).show(10)

# **Dados para pandas**

In [ ]:
df_pandas = df.toPandas()

In [ ]:
df_pandas.head()

In [ ]:
df_pandas.dtypes

In [ ]:
# Importar matplotlib.
import matplotlib.pyplot as plt
# Importar seaborn.
import seaborn as sns

In [ ]:
# Histograma para aas variaveis numéricas.
plt.hist(df_pandas.Magnitude)
plt.xlabel('Magnitude')
plt.ylabel('Frequência')
plt.show()

In [ ]:
plt.boxplot(df_pandas.Magnitude)
plt.show()

In [ ]:
# Plotando dispersão de dados = relacionando duas variáveis numéricas.
plt.figure(figsize = (7,4))
plt.scatter(
    df_pandas['Profundidade(km)'],
    df_pandas['Magnitude'],
    c='blue')

plt.xlabel("Profundidade")
plt.ylabel("Magnitude")
plt.show()

In [ ]:
# Verifcando categorias ou classes
classes_tipos_terremotos = df_pandas['TipoTerremoto'].value_counts()
nome_classes = ['Ke','Sm']
quantidade_classes = [49886,114]
fig = plt.figure(figsize =(7, 7))
plt.pie(quantidade_classes, labels = nome_classes, autopct='%1.2f%%')
plt.show()

In [ ]:
# Gráfico de barras com seaborn
sns.countplot(x='TipoTerremoto', data=df_pandas)
plt.show()

In [ ]:
# Gráfico de barras com seaborn
sns.countplot(x='Mês', data=df_pandas)
plt.show()

In [ ]:
# Gráfico de barras com seaborn
fig = plt.figure(figsize=(15,6))
sns.countplot(x='Ano', data=df_pandas)
plt.show()

In [ ]:
# Gráfico de barras com seaborn
fig = plt.figure(figsize=(15,6))
sns.countplot(x='Ano', data=df_pandas, order = df_pandas["Ano"].value_counts().index)
plt.show()

In [ ]:
# Gráfico de barras com seaborn
sns.countplot(x='Hora_', data=df_pandas)
plt.show()

In [ ]:
# Grafico gerado entre magnitide, profundidade e tipo do terremoto.
sns.relplot(
    data=df_pandas,
    x="Profundidade(km)", y="Magnitude", hue="Ano", palette="rocket_r")
plt.show()

In [ ]:
# Plotar o boxplot de uma variável em relação as classes com seaborn.

plt.subplots( figsize=(8, 6))
sns.boxplot(x='TipoTerremoto', y='Magnitude', data=df_pandas)
plt.show()

In [ ]:
# Plotar o boxplot de uma variável em relação as classes com seaborn.

plt.subplots( figsize=(15, 6))
sns.boxplot(x='Ano', y='Magnitude', data=df_pandas)
plt.show()

# **BASE DE DADOS 2**

In [ ]:
df2 = sc.read.csv('Carros.csv', header=True, sep=";",  inferSchema=True)
df2.show(5)


### String

In [ ]:
from pyspark.sql import functions
print(dir(functions))

In [ ]:
from pyspark.sql.functions import col, lower, upper, substring

In [ ]:
df2.select(col('Carro'),lower(col('Carro')),upper(col('Carro')),substring(col('Carro'),1,4).alias("concatenated value")).show(3, False)

In [ ]:
df2 = df2.withColumn('Carro', upper(df2.Carro)).show(5, False)


### Numérico

In [ ]:
from pyspark.sql.functions import min, max
df2.select(min(df2.Peso), max(df2.Peso)).show()


## SQL

In [ ]:

df = sc.read.csv('Carros.csv', header=True, sep=";")
# Tabela temporária
df.createOrReplaceTempView("temp")

sc.sql("select * from temp limit 15").show()

sc.sql("select count(*) as total_count from temp").show()